In [1]:
""" Import zone """

import re
import html
import datetime

import numpy as np
import pandas as pd
import seaborn as sns

import nltk
from nltk.tokenize import word_tokenize, wordpunct_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer; nltk.download('wordnet')
from nltk.stem.snowball import SnowballStemmer

import spacy; spacy_nlp = spacy.load("en_core_web_sm", disable=['ner', 'entity_linker', 'entity_ruler', 'textcat', 'textcat_multilabel', 'morphologizer', 'senter', 'sentencizer', 'transformer'])

from comodo.comodo import *

import support
register_consts(support.comodo_config)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\tommaso\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tommaso\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Done → COMODO.CONSTANTS in register_consts() → Comodo's costants correctly configured


In [2]:
step = 'clean'
trdf = DF.set_datetime_index(DF.rcsv(step+'/train_df.csv', sep='\t'), 'date')

In [3]:
""" Tengo solo condizioni >= 35"""
keep_condition = list(DICT.dfilter(LIST.occurrences(trdf['condition']), by='value', lambda_fun=lambda v: v>=35).keys())
trdf = DF.find(trdf, {'condition': keep_condition})

In [5]:
""" Raggruppo per condizione """
gr_condition = trdf.groupby('condition')

In [11]:
gr1 = [gr_condition.get_group(condition) for condition in keep_condition[:1]][0]

In [15]:
""" TF-IDF Representation of one condition group """

from sklearn.feature_extraction.text import TfidfVectorizer

tv = TfidfVectorizer(min_df=0., max_df=1., use_idf=True)
tv_matrix = tv.fit_transform(gr1['clean_review'])
tv_matrix = tv_matrix.toarray()

vocab = tv.get_feature_names()
tfidf_df = pd.DataFrame(np.round(tv_matrix, 2), columns=vocab)
tfidf_df

,aa,aaand,ab,abandon,abd,abdomen,abdomin,abdominal,abdominals,abdomonal,...,zoloft,zombie,zomby,zone,zoned,zones,zoom,zovia,zulane,zumba
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28783,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28784,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28785,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28786,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix = cosine_similarity(tv_matrix)
similarity_df = pd.DataFrame(similarity_matrix)
similarity_df

,0,1,2,3,4,5,6,7,8,9,...,28778,28779,28780,28781,28782,28783,28784,28785,28786,28787
0,1.000000,0.081106,0.043651,0.043651,0.012854,0.012854,0.042230,0.042230,0.012751,0.000000,...,0.010562,0.067251,0.125724,0.125724,0.037406,0.037624,0.038830,0.096784,0.128080,0.080385
1,0.081106,1.000000,0.027205,0.027205,0.029276,0.029276,0.028291,0.028291,0.038436,0.000000,...,0.074359,0.085262,0.061104,0.061104,0.047556,0.029508,0.009612,0.071764,0.112467,0.032561
2,0.043651,0.027205,1.000000,1.000000,0.067257,0.067257,0.008653,0.008653,0.000000,0.000000,...,0.019700,0.016885,0.050937,0.050937,0.026262,0.053313,0.007594,0.101164,0.055185,0.036686
3,0.043651,0.027205,1.000000,1.000000,0.067257,0.067257,0.008653,0.008653,0.000000,0.000000,...,0.019700,0.016885,0.050937,0.050937,0.026262,0.053313,0.007594,0.101164,0.055185,0.036686
4,0.012854,0.029276,0.067257,0.067257,1.000000,1.000000,0.010475,0.010475,0.009070,0.000000,...,0.022842,0.030377,0.050977,0.050977,0.022706,0.041482,0.091195,0.035465,0.015576,0.054039
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28783,0.037624,0.029508,0.053313,0.053313,0.041482,0.041482,0.013227,0.013227,0.047669,0.026443,...,0.038646,0.049445,0.080405,0.080405,0.045992,1.000000,0.032318,0.065274,0.070488,0.078127
28784,0.038830,0.009612,0.007594,0.007594,0.091195,0.091195,0.003200,0.003200,0.004347,0.000000,...,0.072841,0.075791,0.056537,0.056537,0.052724,0.032318,1.000000,0.018904,0.052018,0.021008
28785,0.096784,0.071764,0.101164,0.101164,0.035465,0.035465,0.010263,0.010263,0.092141,0.000000,...,0.037401,0.144565,0.068353,0.068353,0.076780,0.065274,0.018904,1.000000,0.079896,0.104090
28786,0.128080,0.112467,0.055185,0.055185,0.015576,0.015576,0.008454,0.008454,0.030617,0.045094,...,0.051338,0.071405,0.116349,0.116349,0.050987,0.070488,0.052018,0.079896,1.000000,0.150852
